In [1]:
!pip install chainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.4 MB/s eta 0:00:0000:0100:010m
  Preparing metadata (setup.py) ... done
  Created wheel for chainer: filename=chainer-7.8.1-py3-none-any.whl size=971824 sha256=23000afc75219ffc06e9370d997f9d1d517ffb9105daf0d1ead45069d27a5921
  Stored in directory: /root/.cache/pip/wheels/c4/95/6a/16014db6f761c4e742755b64aac60dbe142da1df6c5919f790
Successfully built chainer


<p> The chainer import warns that the cupy version might be incompatible, but installing the stated cupy version leads to other conflicts down the line. Installing all the packages from the QCANet requirements.txt does not help either because it fails with errors. Some of the package versions are so old that they're no longer available. </p>

In [2]:
import chainer
from chainer import cuda

import csv
import sys
sys.path.insert(1, "/kaggle/input/qcanet3/QCANet")
import time
import random
import copy
import math
import os
import numpy as np
import argparse
import configparser

sys.path.append(os.getcwd())
from src.lib.trainer import NSNTrainer, NDNTrainer
from src.lib.utils import createOpbase
from src.lib.utils import create_dataset_parser, create_model_parser, create_runtime_parser
from src.lib.utils import print_args
from src.lib.utils import get_dataset, get_model

/opt/conda/lib/python3.10/site-packages/chainer/_environment_check.py:72: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy) version 13.0.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  warnings.warn(msg.format(


In [3]:
from src.lib.model import Model_L4
class_weight = np.array([1, 1]).astype(np.float32)
ndn = Model_L4(class_weight=class_weight, n_class=2, init_channel=12,
                   kernel_size=5, pool_size=2, ap_factor=2, gpu=0)
cuda.get_device_from_id(0).use()  # Make a specified GPU current
ndn.to_gpu()  # Copy the SegmentNucleus model to the GPU

<p> Usually people let their dataset classes handle their test-train split for them, but for some reason, the authors of QCANet ask for separate text files that contain the filenames of the training images and testing images respectively. I was not able to get it to find the right text files on Kaggle because it got confused with the relative paths.</p>

In [8]:
from src.lib.dataset import PreprocessedDataset
train_dataset = PreprocessedDataset(
    root_path="/kaggle/input/qcanet-dataset/qcanet-dataset/images",
    split_list="/kaggle/input/qcanet-dataset/qcanet-dataset/train_split.txt",
    train=True,
    model='NDN',
    arr_type='tif'
)
test_dataset = PreprocessedDataset(
    root_path="/kaggle/input/qcanet-dataset/qcanet-dataset/images",
    split_list="/kaggle/input/qcanet-dataset/qcanet-dataset/test_split.txt",
    train=False,
    model='NDN'
)
train_iterator = chainer.iterators.SerialIterator(
    train_dataset, 1, repeat=True, shuffle=True
)
test_iterator = chainer.iterators.SerialIterator(
    test_dataset, 1, repeat=True, shuffle=False
)

FileNotFoundError: [Errno 2] No such file or directory: 'train_split.txt'

In [5]:
from src.lib.utils import createOpbase
opbase = createOpbase("/kaggle/working")
with open('/kaggle/working/TestResult.csv', 'w') as f:
    c = csv.writer(f)
    c.writerow(['Epoch', 'Accuracy', 'Recall', 'Precision', 'Specificity', 'F-measure', 'IoU'])
with open('/kaggle/working/TrainResult.csv', 'w') as f:
    c = csv.writer(f)
    c.writerow(['Epoch', 'Accuracy', 'Recall', 'Precision', 'Specificity', 'F-measure', 'IoU'])

Output Directory not exist! Create...
Output Directory: /kaggle/working_Apr9Tue_2024_090559


In [6]:
trainer = NDNTrainer(
    model=ndn,
    epoch=1,
    patchsize=(128, 128, 128),
    batchsize=1,
    gpu=0,
    opbase=opbase,
)

<p> The experiment ended here because it always struggled to find at least one of the text files with the filenames. For the files that it did find, it would throw errors related to the input dimensions of the layer, which is weird because the PreprocessedDataset class was supposed to take care of that. Other times, it would throw an error because somewhere in the training loop, they used a function which is now deprecated in the current version of skimage. Downgrading to their version of skimage broke other things. Also, certain Kaggle GPUs caused errors, something about CUDA instruction sets. </p>

In [7]:
train_eval, test_eval, best_score = trainer.training((train_iterator, test_iterator))

[epoch 1]


/opt/conda/lib/python3.10/site-packages/chainer/functions/connection/convolution_nd.py:174: PerformanceWarning: The best algo of conv fwd might not be selected due to lack of workspace size (8388608)
  cuda.cudnn.convolution_forward(


FileNotFoundError: No such file: '/kaggle/input/qcanet-dataset/qcanet-dataset/images/images_raw/train_split.txt'